In [1]:
import elli
from elli.fitting import ParamsHist, fit
from elli.importer.spectraray import read_spectraray

# Load data
Load data collected with Sentech Ellipsometer and cut the spectral range (to use Si Aspnes file)

The sample is an ALD grown TiO2 sample (with 400 cycles) on commercially available SiO2 / Si substrate.

In [2]:
data = read_spectraray("test_TiO2/TiO2_400cycles.txt").sel(Angle_of_Incidence=70.06, Wavelength=slice(400, 800))
data

<xarray.Dataset> Size: 37kB
Dimensions:             (Wavelength: 926)
Coordinates:
    Angle_of_Incidence  float64 8B 70.06
  * Wavelength          (Wavelength) float64 7kB 400.1 400.5 ... 799.3 799.7
Data variables:
    psi                 (Wavelength) float64 7kB 32.76 32.84 ... 20.22 20.18
    delta               (Wavelength) float64 7kB -132.6 -132.1 ... -117.8 -117.7
    rho                 (Wavelength) complex128 15kB (-0.43531353819854135+0....

# Estimate Parameters and build model

In [3]:
params = ParamsHist()
params.add("SiO2_n0", value=1.452, min=-100, max=100, vary=False)
params.add("SiO2_n1", value=36.0, min=-40000, max=40000, vary=False)
params.add("SiO2_n2", value=0, min=-40000, max=40000, vary=False)
params.add("SiO2_k0", value=0, min=-100, max=100, vary=False)
params.add("SiO2_k1", value=0, min=-40000, max=40000, vary=False)
params.add("SiO2_k2", value=0, min=-40000, max=40000, vary=False)
params.add("SiO2_d", value=276.36, min=0, max=40000, vary=False)

params.add("TiO2_n0", value=2.236, min=-100, max=100, vary=True)
params.add("TiO2_n1", value=451, min=-40000, max=40000, vary=True)
params.add("TiO2_n2", value=251, min=-40000, max=40000, vary=True)
params.add("TiO2_k0", value=0, min=-100, max=100, vary=False)
params.add("TiO2_k1", value=0, min=-40000, max=40000, vary=False)
params.add("TiO2_k2", value=0, min=-40000, max=40000, vary=False)

params.add("TiO2_d", value=20, min=0, max=40000, vary=True)

In [4]:
@fit(data, params)
def model(lbda, params):
    sr = elli.TableSpectraRay("./")
    Si = elli.IsotropicMaterial(sr.load_dispersion_table("test_TiO2/Si_Aspnes.mat"))

    SiO2 = elli.Cauchy(
        params["SiO2_n0"],
        params["SiO2_n1"],
        params["SiO2_n2"],
        params["SiO2_k0"],
        params["SiO2_k1"],
        params["SiO2_k2"],
    ).get_mat()
    TiO2 = elli.Cauchy(
        params["TiO2_n0"],
        params["TiO2_n1"],
        params["TiO2_n2"],
        params["TiO2_k0"],
        params["TiO2_k1"],
        params["TiO2_k2"],
    ).get_mat()

    Layer = [elli.Layer(TiO2, params["TiO2_d"]), elli.Layer(SiO2, params["SiO2_d"])]

    return elli.Structure(elli.AIR, Layer, Si).evaluate(lbda, 70, solver=elli.Solver2x2)

# Fit to experimental data

In [5]:
out = model.fit()
out

# Show fits

In [6]:
model.plot()

FigureWidget({
    'data': [{'hovertemplate': 'variable=psi<br>index=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'psi',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'psi',
              'showlegend': True,
              'type': 'scattergl',
              'uid': '56cdb61e-f0c5-4570-85f4-848db160625d',
              'x': array([400.07646, 400.51975, 400.96301, ..., 798.88197, 799.30046, 799.71891]),
              'xaxis': 'x',
              'y': array([32.75947, 32.84076, 32.84675, ...,      nan,      nan,      nan]),
              'yaxis': 'y'},
             {'hovertemplate': 'variable=delta<br>index=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'delta',
              'line': {'color': '#EF553B', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'delta',
              

In [7]:
model.plot_rho()

FigureWidget({
    'data': [{'hovertemplate': 'variable=ρr<br>index=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'ρr',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'ρr',
              'showlegend': True,
              'type': 'scattergl',
              'uid': '4c778467-92de-4a41-9ca0-a40ade67d458',
              'x': array([400.07646, 400.51975, 400.96301, ..., 798.88197, 799.30046, 799.71891]),
              'xaxis': 'x',
              'y': array([-0.43531354, -0.43287997, -0.42791407, ..., -0.17317231, -0.17166007,
                          -0.17089142]),
              'yaxis': 'y'},
             {'hovertemplate': 'variable=ρi<br>index=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'ρi',
              'line': {'color': '#EF553B', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
          